<a href="https://colab.research.google.com/github/c-etulle/Vegetation-dynamics-in-the-Subandean-grasslands-of-Chubut/blob/main/4_Merge_datasets_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For merging SPEI, NDVI, MODIS and the clusters.

# Packages

In [ ]:
#!pip install rasterio
%pip install unidecode

import os, sys
from google.colab import drive
import glob
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from unidecode import unidecode
import re
from datetime import datetime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 3.2 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/ee-meinzinger-patagonia/Data

Mounted at /content/drive
/content/drive/My Drive/ee-meinzinger-patagonia/Data


# Define faulty SPEI-locations

In [ ]:
# Create a list of faulty SPEI locations
faulty_locations = [
    "Cerro Condor",
    "La Clara",
    "Berna",
    "Bahia Cracker",
    "Valdes Creek Secc La Adela",
    "La Esperanza Biedma"
]

# Import Landsat and MODIS

## a) Landsat - import as "combined_df"

Read all csv-files (for different site names) and combine into one data frame

In [ ]:
# find all the files that end with '_NDVI.csv'
path = "landsat-2000-2024"

csv_files = glob.glob(os.path.join(path, '*_NDVI.csv'))

print(csv_files)

['landsat-2000-2024/La Ana_NDVI.csv', 'landsat-2000-2024/CERM_NDVI.csv', 'landsat-2000-2024/Alto Rio Mayo_NDVI.csv', 'landsat-2000-2024/1 San Felipe_NDVI.csv', 'landsat-2000-2024/Cerro Condor_NDVI.csv', 'landsat-2000-2024/2 San Felipe_NDVI.csv', 'landsat-2000-2024/La Isabel_NDVI.csv', 'landsat-2000-2024/La Clara_NDVI.csv', 'landsat-2000-2024/Berna_NDVI.csv', 'landsat-2000-2024/Las Piedritas_NDVI.csv', 'landsat-2000-2024/Bahia Cracker_NDVI.csv', 'landsat-2000-2024/San Sebastian_NDVI.csv', 'landsat-2000-2024/Bella Vista_NDVI.csv', 'landsat-2000-2024/La Regina_NDVI.csv', 'landsat-2000-2024/La Payanca_NDVI.csv', 'landsat-2000-2024/Los 5 Hermanos_NDVI.csv', 'landsat-2000-2024/Cerco de Piedras_NDVI.csv', 'landsat-2000-2024/Chacra Berwyn_NDVI.csv', 'landsat-2000-2024/El Moro_NDVI.csv', 'landsat-2000-2024/1 San Jose_NDVI.csv', 'landsat-2000-2024/Don Julian_NDVI.csv', 'landsat-2000-2024/2 San Jose_NDVI.csv', 'landsat-2000-2024/La Portena_NDVI.csv', 'landsat-2000-2024/La Juanita_NDVI.csv', 'land

In [ ]:
# Make a placeholder-list
df_list = []

# Loop through the list of csv-files and read them into dfs
for file in csv_files:
  df = pd.read_csv(file)
  df['Site_Name'] = file # make a new column in the individual df
  df_list.append(df) # indivial dfs are getting "added together"

# Concatenate all dfs into a single df
landsat_df = pd.concat(df_list, ignore_index=True)

# Remove random column called .geo
landsat_df = landsat_df.drop('.geo', axis=1)

# Remove Site_Name, since it is the same in all of them
landsat_df['Site_Name'] = landsat_df['Site_Name'].str.replace("_NDVI.csv", '', regex=False)

# Remove prefix, since it is the same in all of them
landsat_df['Site_Name'] = landsat_df['Site_Name'].str.replace("landsat-2000-2024/", '', regex=False)

# Re-scale NDVI-values
landsat_df['NDVI'] = landsat_df['NDVI']/10000

# Define month & year as int
landsat_df['year'] = landsat_df['year'].astype(int)
landsat_df['month'] = landsat_df['month'].astype(int)

# Adds a column called year_month
landsat_df['year_month'] = landsat_df['year'].astype(str) + '_' + landsat_df['month'].apply(lambda x: f"{x:02d}") #puts a 0 in front of all the months until september

# Rename NDVI column
landsat_df = landsat_df.rename(columns={'NDVI': 'NDVI_Landsat'})

# Remove the 'system:index' column
landsat_df = landsat_df.drop('system:index', axis=1)

# Remove rows where 'Site_Name' is in Faulty_locations
landsat_df = landsat_df[~landsat_df['Site_Name'].isin(faulty_locations)]

# Remove blank spaces
landsat_df['Site_Name'] = landsat_df['Site_Name'].str.rstrip()

# Number of sites
len(landsat_df["Site_Name"].unique())

97

In [ ]:
print(landsat_df.head())

print("dimensions: " + str(landsat_df.shape))

   NDVI_Landsat  month  year Site_Name year_month
0        0.0692      1  2000    La Ana    2000_01
1        0.0552      2  2000    La Ana    2000_02
2        0.0491      3  2000    La Ana    2000_03
3        0.0460      4  2000    La Ana    2000_04
4        0.0215      5  2000    La Ana    2000_05
dimensions: (29100, 5)


## b) MODIS - import as "combined_df_modis"

In [ ]:
path_modis = "modis-2000-2024"

# find all the files that end with '_NDVI.csv'
csv_files_modis = glob.glob(os.path.join(path_modis, '*_NDVI.csv'))

## Read all csv-files and combine them into one df

# Make a placeholder-list
df_list_modis = []

# Loop through the list of csv-files and read them into dfs
for file in csv_files_modis:
  df = pd.read_csv(file)
  df['Site_Name'] = file # make a new column in the individual df
  df_list_modis.append(df) # indivial dfs are getting "added together"

# Concatenate all dfs into a single df
modis_df = pd.concat(df_list_modis, ignore_index=True)

# Remove random column called .geo
modis_df = modis_df.drop('.geo', axis=1)

# Remove suffix, since it is the same in all of them
modis_df['Site_Name'] = modis_df['Site_Name'].str.replace("_NDVI.csv", '', regex=False)

# Remove prefix, since it is the same in all of them
modis_df['Site_Name'] = modis_df['Site_Name'].str.replace("modis-2000-2024/", '', regex=False)

# Re-scale NDVI-values
modis_df['NDVI'] = modis_df['NDVI']/10000

# Define month & year as int
modis_df['year'] = modis_df['year'].astype(int)
modis_df['month'] = modis_df['month'].astype(int)

# Adds a column called year_month
modis_df['year_month'] = modis_df['year'].astype(str) + '_' + modis_df['month'].apply(lambda x: f"{x:02d}") #puts a 0 in front of all the months until september

# Remove the 'system:index' column
modis_df = modis_df.drop('system:index', axis=1)

# Rename NDVI column
modis_df = modis_df.rename(columns={'NDVI': 'NDVI_MODIS'})

# Remove rows where 'Site_Name' is in Faulty_locations
modis_df = modis_df[~modis_df['Site_Name'].isin(faulty_locations)]

# Remove blank spaces
modis_df['Site_Name'] = modis_df['Site_Name'].str.rstrip()

# Number of sites
len(modis_df["Site_Name"].unique())

97

In [ ]:
print(modis_df.head())

print("dimensions: " + str(modis_df.shape))

   NDVI_MODIS  month  year Site_Name year_month
0         NaN      1  2000      CERM    2000_01
1     0.15530      2  2000      CERM    2000_02
2     0.15225      3  2000      CERM    2000_03
3     0.15715      4  2000      CERM    2000_04
4     0.15955      5  2000      CERM    2000_05
dimensions: (29100, 5)


## c) Import SPEI-data




In [ ]:
# Import spei.csv files
file_path = '../pts_drought.csv'
spei_data = pd.read_csv(file_path)

## Restructuring the df

# Function to transform the column names, so instead of "YYYYMM", you get "YYYY_MM"
def transform_column_name(col_name):
    if col_name.isdigit() and len(col_name) == 6:
        return col_name[:4] + "_" + col_name[4:]
    return col_name

# Apply the transformation to the columns
spei_data.columns = [transform_column_name(col) for col in spei_data.columns]

# Set 'Site_Name' as the index and then transpose the DataFrame
spei_data = spei_data.set_index('Site_Name').transpose()

# Function to clean site names: remove (, ), and °, and normalize special characters
def clean_site_name(site_name):
    # Remove (, ), and °
    site_name = re.sub(r'[()°]', '', site_name)
    # Convert special characters to their closest ASCII equivalents
    site_name = unidecode(site_name)
    return site_name

# Apply the cleaning function to the column names
spei_data.columns = [clean_site_name(col) for col in spei_data.columns]

# Remove duplicates (San Felipe & San José)
# spei_data = spei_data.loc[:, ~spei_data.columns.duplicated()]
# spei_data.head(12)

# Remove faulty locations
spei_data = spei_data.drop(columns=faulty_locations)

SPEI_df_reset = spei_data.reset_index().rename(columns={'index': 'year_month'})

SPEI_long = pd.melt(SPEI_df_reset, id_vars=['year_month'], var_name='Site_Name', value_name='SPEI')


# Split 'year_month' into 'year' and 'month'
SPEI_long[['year', 'month']] = SPEI_long['year_month'].str.split('_', expand=True)

# Convert 'year' and 'month' to integers (if necessary)
SPEI_long['year'] = SPEI_long['year'].astype(int)
SPEI_long['month'] = SPEI_long['month'].astype(int)

# Remove blank spaces
SPEI_long['Site_Name'] = SPEI_long['Site_Name'].str.rstrip()

len(SPEI_long["Site_Name"].unique())


97

In [ ]:
print(SPEI_long.head())

print("dimensions: " + str(SPEI_long.shape))

  year_month Site_Name      SPEI  year  month
0    2000_01      CERM -0.813071  2000      1
1    2000_02      CERM -0.531295  2000      2
2    2000_03      CERM -0.411697  2000      3
3    2000_04      CERM  0.126315  2000      4
4    2000_05      CERM  0.385927  2000      5
dimensions: (26772, 5)


## d) Import MARAS data

In [ ]:
# Load the CSV file into a DataFrame
maras_chubut = pd.read_csv('MARAS_Chubut_utf8.csv', sep=';')

# Convert the 'Date' column to datetime format
maras_chubut['Date'] = pd.to_datetime(maras_chubut['Date'], format='%d-%m-%Y')

# Create the 'year_month' column in the format YYYY_MM
maras_chubut['year_month'] = maras_chubut['Date'].dt.strftime('%Y_%m')

# Remove (, ), and ° using re.sub with apply
maras_chubut["Site_Name"] = maras_chubut["Site_Name"].apply(lambda x: re.sub(r'[()°]', '', x))

# Convert special characters to their closest ASCII equivalents
maras_chubut["Site_Name"] = maras_chubut["Site_Name"].apply(unidecode)

# Remove rows where 'Site_Name' is in Faulty_locations
maras_chubut = maras_chubut[~maras_chubut['Site_Name'].isin(faulty_locations)]

# Split 'year_month' into 'year' and 'month'
maras_chubut[['year', 'month']] = maras_chubut['year_month'].str.split('_', expand=True)

# Convert 'year' and 'month' to integers (if necessary)
maras_chubut['year'] = maras_chubut['year'].astype(int)
maras_chubut['month'] = maras_chubut['month'].astype(int)

# Remove blank spaces
maras_chubut['Site_Name'] = maras_chubut['Site_Name'].str.rstrip()
maras_chubut['Biozone'] = maras_chubut['Biozone'].str.rstrip()

# Number of sites
len(maras_chubut["Site_Name"].unique())

97

In [ ]:
print(maras_chubut.head())

print("dimensions: " + str(maras_chubut.shape))

     Country Province Site_Name       ID  DB_code  Lectura       Date  \
0  Argentina   Chubut      CERM  #CH-001      301        1 2008-06-18   
1  Argentina   Chubut      CERM  #CH-001      301        2 2014-01-21   
2  Argentina   Chubut      CERM  #CH-001      301        3 2018-12-14   
3  Argentina   Chubut    La Ana  #CH-002      302        1 2009-04-21   
4  Argentina   Chubut    La Ana  #CH-002      302        2 2014-01-24   

     Latitude   Longitude  Altitude  ... InterP_pH  InterP_OC InterP_N  \
0  -45,440611  -70,300306       543  ...        NaN       NaN      NaN   
1  -45,440611  -70,300306       543  ...        6,9      0,53     0,04   
2  -45,440611  -70,300306       543  ...        6,7      0,19     0,05   
3   -45,42675  -70,267778       535  ...        NaN       NaN      NaN   
4   -45,42675  -70,267778       535  ...        6,9       0,7     0,05   

  InterP_OM InterP_Clay InterP_Silt  InterP_Sand year_month  year month  
0       NaN         NaN         NaN       

## e) Make the biozone data frame

To have the Biozone in the beginning of the data frame

In [ ]:
Biozones = maras_chubut[["Site_Name", "Biozone"]].drop_duplicates()

# Merge all

In [ ]:
all_data = landsat_df.\
merge(modis_df, on = ["Site_Name", "year_month", "month", "year"], how = "outer").\
merge(SPEI_long, on = ["Site_Name", "year_month", "month", "year"], how = "outer").\
merge(maras_chubut.drop('Biozone', axis=1), on = ["Site_Name", "year_month", "month", "year"], how = "outer").\
merge(Biozones, on = ["Site_Name"], how = "outer")

In [ ]:
all_data.head()

,NDVI_Landsat,month,year,Site_Name,year_month,NDVI_MODIS,SPEI,Country,Province,ID,...,Patch_Sand,InterP_Con,InterP_pH,InterP_OC,InterP_N,InterP_OM,InterP_Clay,InterP_Silt,InterP_Sand,Biozone
0,0.04255,1,2000,1 Rio Guenguel,2000_01,NaN,-1.202183,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Plateaus Shrublands
1,0.04015,2,2000,1 Rio Guenguel,2000_02,0.14080,-0.862369,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Plateaus Shrublands
2,0.03890,3,2000,1 Rio Guenguel,2000_03,0.13175,-0.742418,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Plateaus Shrublands
3,0.04640,4,2000,1 Rio Guenguel,2000_04,0.14920,-0.189822,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Plateaus Shrublands
4,NaN,5,2000,1 Rio Guenguel,2000_05,0.13650,-0.093864,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,West Plateaus Shrublands


## Reorder

In [ ]:
# Define the desired order for specific columns
desired_order = ['Site_Name', 'year_month', 'year', 'month', 'NDVI_Landsat', 'NDVI_MODIS', 'SPEI', "Biozone"]

# Get the list of other columns that are not in the desired order
other_columns = [col for col in all_data.columns if col not in desired_order]

# Reorganize the DataFrame
all_data = all_data[desired_order + other_columns]

# Convert the 'year' column to integers
all_data["year"] = all_data["year"].astype(int)

# Convert the 'year' column to integers
all_data["month"] = all_data["month"].astype(int)


# Save as csv

Uncheck .to_csv line in order to actually save the csv

In [ ]:
print(all_data.head())

# Get the current date and time
current_time = datetime.now()

# Format the date and time
formatted_time = current_time.strftime("%Y%m%d_%H%M")

# Construct the filename
filename = f'Merged_Data_{formatted_time}.csv'

# Save the DataFrame as a CSV file
# all_data.to_csv(filename, index=False)

print(f"Data saved as: {filename}")

        Site_Name year_month  year  month  NDVI_Landsat  NDVI_MODIS      SPEI  \
0  1 Rio Guenguel    2000_01  2000      1       0.04255         NaN -1.202183   
1  1 Rio Guenguel    2000_02  2000      2       0.04015     0.14080 -0.862369   
2  1 Rio Guenguel    2000_03  2000      3       0.03890     0.13175 -0.742418   
3  1 Rio Guenguel    2000_04  2000      4       0.04640     0.14920 -0.189822   
4  1 Rio Guenguel    2000_05  2000      5           NaN     0.13650 -0.093864   

                    Biozone Country Province  ... Patch_Silt  Patch_Sand  \
0  West Plateaus Shrublands     NaN      NaN  ...        NaN         NaN   
1  West Plateaus Shrublands     NaN      NaN  ...        NaN         NaN   
2  West Plateaus Shrublands     NaN      NaN  ...        NaN         NaN   
3  West Plateaus Shrublands     NaN      NaN  ...        NaN         NaN   
4  West Plateaus Shrublands     NaN      NaN  ...        NaN         NaN   

   InterP_Con InterP_pH  InterP_OC InterP_N  InterP_OM I